In [21]:
import pickle
import itertools
import numpy as np
from scipy.stats import ttest_ind

In [20]:
pickle_in = open("errors.p","rb")
errors = pickle.load(pickle_in)

In [23]:
perms = itertools.combinations(errors.keys(), r=2)
for perm in perms:
    a1 = errors[perm[0]].astype(np.float)
    a2 = errors[perm[1]].astype(np.float)
    t_stat, p_val = ttest_ind(a1,a2)
    print(t_stat,p_val )

-0.1851344254977115 0.8532799572802177
0.43289606125573965 0.6654790495967557
-0.23813787127094255 0.811977352113328
-0.009317187098025361 0.9925738078939808
0.6368423876509429 0.5248344770090425
-0.07520884381232948 0.9401111993427456
0.17516663808131008 0.8610963876865952
-0.6246577273000408 0.5327891122369554
-0.4411892626637381 0.6594731767196389
0.22930291675275183 0.8188287764555359
